#1. Import packages

In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

# first install yahoo finance
!pip install yfinance
!pip install yahoofinancials
import yfinance as yf
from yahoofinancials import YahooFinancials
!pip install stockstats # used to calculate technical indicators
import stockstats

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# 

#2. Download Financial Data

In [39]:
data = yf.download(
  tickers = "AXP AMGN AAPL BA CAT CSCO CVX GS HD HON IBM INTC JNJ KO JPM MCD MMM MRK MSFT NKE PG TRV UNH CRM VZ V WBA WMT DIS DOW", # Dow Jones 30 Tickers
  start="2008-01-01",
  end = "2022-11-30",
  interval="1d",
  ignore_tz=True,
  group_by='ticker',
  auto_adjust= True,
  repair=True,
  prepost=False,
  threads=True,
  proxy=None
)

[*********************100%***********************]  30 of 30 completed


In [40]:
data.head()

JPM                                                   MCD  \
                 Open       High        Low      Close    Volume       Open   
Date                                                                          
2008-01-02  29.878320  30.042978  28.814913  28.931543  22122300  38.888742   
2008-01-03  29.048185  29.260867  28.650263  28.732594  17150700  38.182647   
2008-01-04  28.410131  28.677698  27.874998  28.080818  25873800  37.483057   
2008-01-07  28.197460  28.526776  27.552555  28.362118  25644100  37.502667   
2008-01-08  28.444449  28.526779  27.044866  27.236967  33646000  38.208789   

                                                       ...        TRV  \
                 High        Low      Close    Volume  ...       Open   
Date                                                   ...              
2008-01-02  38.888742  37.849181  37.986481   7858300  ...  37.233774   
2008-01-03  38.444171  37.751131  37.875355   6514700  ...  36.331725   
2008-01-04  37.574590  36.894625  37.299988   9687500  ...  36.512117   
2008-01-07  38.084559  37.182298  37.940720  10784500  ...  35.936206   
2008-01-08  38.339552  37.241148  37.319607  10080800  ...  36.220700   

                                                           MSFT             \
                 High        Low      Close   Volume       Open       High   
Date                                                                         
2008-01-02  37.261530  36.179071  36.331726  2341400  26.274274  26.399073   
2008-01-03  37.004793  36.303969  36.817444  2098800  25.855810  26.171483   
2008-01-04  36.817428  35.617011  35.658642  3048900  25.833791  25.841134   
2008-01-07  36.505190  35.401915  36.109676  3130300  25.363945  25.547476   
2008-01-08  36.519070  34.846812  35.006405  4684600  25.481409  25.481409   

                                            
                  Low      Close    Volume  
Date                                        
2008-01-02  25.694315  25.855824  63004200  
2008-01-03  25.591526  25.965927  49599600  
2008-01-04  25.026257  25.239153  72090800  
2008-01-07  25.143709  25.407993  80164300  
2008-01-08  24.519710  24.556416  79148300  

[5 rows x 150 columns]

In [41]:
data.shape

(3755, 150)

There is OHLCV (Open, High, Low, Close, Volume) data for 30 stocks = 150 columns, and 3502 days (rows)

# 3. Preprocess Data

Important steps:

1. Check for missing data

2. Add chosen technical indicators to use as features

In [42]:
data_copy = data.copy()

Check for missing values

In [43]:
data_copy.isnull().sum()[data_copy.isnull().sum() != 0]

DOW  Open      2822
     High      2822
     Low       2822
     Close     2822
     Volume    2822
V    Open        53
     High        53
     Low         53
     Close       53
     Volume      53
dtype: int64

DOW and V are missing significant number of days -> drop them from the dataset, leaving us with 28 stocks

In [44]:
ticker_list = ['AXP', 'AMGN', 'AAPL', 'BA', 'CAT', 'CSCO', 'CVX', 'GS', 'HD', 
               'HON', 'IBM', 'INTC', 'JNJ', 'KO', 'JPM', 'MCD', 'MMM', 'MRK', 
               'MSFT', 'NKE', 'PG', 'TRV', 'UNH', 'CRM', 'VZ', 'WBA', 'WMT', 
               'DIS'] # dropped V and DOW

In [75]:
from stockstats import wrap

def add_indicators_reshape_df(df, ticker_list, indicator_list):
  stock_df_list = []
  for i in ticker_list:
    temp = df[i]
    temp['tick'] = i

    # get and add indicators to the dataframe
    temp_w = wrap(temp)
    t = temp.merge(temp_w[indicator_list], how="left")
    t.set_index(temp.index, inplace=True)

    stock_df_list.append(t)

  new_df = pd.concat(stock_df_list, sort=False).sort_index()
  return new_df

In [78]:
data_w_ind = add_indicators_reshape_df(data_copy, ticker_list, indicator_list=['macd', 'rsi', 'cci'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


Check for missing values

In [79]:
data_w_ind.head()

,open,high,low,close,volume,tick,macd,macds,macdh,rs_14,rsi,cci
Date,,,,,,,,,,,,
2008-01-02,40.999968,41.181001,39.976741,40.173515,8053700,AXP,0.0,0.0,0.0,NaN,NaN,NaN
2008-01-02,64.177873,64.375747,63.027259,63.481644,4303000,BA,0.0,0.0,0.0,NaN,NaN,NaN
2008-01-02,27.419730,27.682730,26.885249,27.012506,9269900,DIS,0.0,0.0,0.0,NaN,NaN,NaN
2008-01-02,47.810793,47.883274,46.156923,46.539089,6337800,CAT,0.0,0.0,0.0,NaN,NaN,NaN
2008-01-02,19.173505,19.386543,18.612501,18.846846,64338900,CSCO,0.0,0.0,0.0,NaN,NaN,NaN


In [84]:
data_w_ind[data_w_ind.isna().any(axis=1)]

,open,high,low,close,volume,tick,macd,macds,macdh,rs_14,rsi,cci
Date,,,,,,,,,,,,
2008-01-02,40.999968,41.181001,39.976741,40.173515,8053700,AXP,0.0,0.0,0.0,NaN,NaN,NaN
2008-01-02,64.177873,64.375747,63.027259,63.481644,4303000,BA,0.0,0.0,0.0,NaN,NaN,NaN
2008-01-02,27.419730,27.682730,26.885249,27.012506,9269900,DIS,0.0,0.0,0.0,NaN,NaN,NaN
2008-01-02,47.810793,47.883274,46.156923,46.539089,6337800,CAT,0.0,0.0,0.0,NaN,NaN,NaN
2008-01-02,19.173505,19.386543,18.612501,18.846846,64338900,CSCO,0.0,0.0,0.0,NaN,NaN,NaN
2008-01-02,52.506486,52.914425,51.807964,52.227077,9058000,CVX,0.0,0.0,0.0,NaN,NaN,NaN
2008-01-02,172.324138,172.524702,165.745659,166.547913,9573500,GS,0.0,0.0,0.0,NaN,NaN,NaN
2008-01-02,34.043740,34.201716,33.361572,33.677525,19918600,WMT,0.0,0.0,0.0,NaN,NaN,NaN
2008-01-02,18.665301,18.761621,17.901627,17.963547,16560700,HD,0.0,0.0,0.0,NaN,NaN,NaN


NaN values for first two days in the dataset, therefore drop these days.

In [89]:
df = data_w_ind.copy()

df = df.drop([pd.to_datetime('2008-01-02'), pd.to_datetime('2008-01-03')])
df

,open,high,low,close,volume,tick,macd,macds,macdh,rs_14,rsi,cci
Date,,,,,,,,,,,,
2008-01-04,42.311006,42.748795,42.311006,42.388260,12714800,JNJ,-0.001429,-0.000491,-0.000938,0.206125,17.089854,-24.839588
2008-01-04,46.034172,46.862402,46.034172,46.239624,10049000,PG,-0.005774,-0.002366,-0.003407,0.000000,0.000000,-100.000000
2008-01-04,53.385068,53.814755,53.092104,53.228821,3620300,MMM,-0.019163,-0.007902,-0.011261,0.000000,0.000000,-100.000000
2008-01-04,36.512117,36.817428,35.617011,35.658642,3048900,TRV,-0.022049,-0.005463,-0.016585,0.389215,28.016925,-100.000000
2008-01-04,26.299863,26.876764,26.299863,26.410152,9550700,DIS,-0.018535,-0.008033,-0.010502,0.000000,0.000000,-100.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-29,531.669983,532.969971,523.650024,528.000000,3751000,UNH,-0.417645,-0.561892,0.144247,0.991349,49.782786,15.513895
2022-11-29,152.720001,154.020004,151.259995,151.679993,6398100,CRM,-0.493688,-0.601808,0.108121,0.964399,49.093839,-0.847816
2022-11-29,38.020000,38.369999,37.950001,38.340000,14008800,VZ,0.272868,0.201219,0.071649,1.096433,52.299936,-34.559979


Make date a column and make a column called day that starts at 1 and increments upwards.

In [90]:
df_c = df.copy()
df_c['Date'] = df.index

df_c['day'] = 1
for i in range(1,df_c.shape[0]):
  if df_c.iloc[i,-2] == df_c.iloc[i-1,-2]:
    df_c.iloc[i]

df_c

,open,high,low,close,volume,tick,macd,macds,macdh,rs_14,rsi,cci,Date
Date,,,,,,,,,,,,,
2008-01-04,42.311006,42.748795,42.311006,42.388260,12714800,JNJ,-0.001429,-0.000491,-0.000938,0.206125,17.089854,-24.839588,2008-01-04
2008-01-04,46.034172,46.862402,46.034172,46.239624,10049000,PG,-0.005774,-0.002366,-0.003407,0.000000,0.000000,-100.000000,2008-01-04
2008-01-04,53.385068,53.814755,53.092104,53.228821,3620300,MMM,-0.019163,-0.007902,-0.011261,0.000000,0.000000,-100.000000,2008-01-04
2008-01-04,36.512117,36.817428,35.617011,35.658642,3048900,TRV,-0.022049,-0.005463,-0.016585,0.389215,28.016925,-100.000000,2008-01-04
2008-01-04,26.299863,26.876764,26.299863,26.410152,9550700,DIS,-0.018535,-0.008033,-0.010502,0.000000,0.000000,-100.000000,2008-01-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-29,531.669983,532.969971,523.650024,528.000000,3751000,UNH,-0.417645,-0.561892,0.144247,0.991349,49.782786,15.513895,2022-11-29
2022-11-29,152.720001,154.020004,151.259995,151.679993,6398100,CRM,-0.493688,-0.601808,0.108121,0.964399,49.093839,-0.847816,2022-11-29
2022-11-29,38.020000,38.369999,37.950001,38.340000,14008800,VZ,0.272868,0.201219,0.071649,1.096433,52.299936,-34.559979,2022-11-29


In [108]:
df_a = df_c.copy()
df_a['Day'] = df_a['Date']
df_a['Day'].replace(list(df_a['Day'].unique()), np.arange(0, len(list(df_a['Day'].unique()))), inplace=True)

In [109]:
df_a = df_a.reset_index(drop=True)

In [117]:
df_a = df_a.set_index(df_a['Day'])

In [118]:
df_a

,open,high,low,close,volume,tick,macd,macds,macdh,rs_14,rsi,cci,Date,Day
Day,,,,,,,,,,,,,,
0,42.311006,42.748795,42.311006,42.388260,12714800,JNJ,-0.001429,-0.000491,-0.000938,0.206125,17.089854,-24.839588,2008-01-04,0
0,46.034172,46.862402,46.034172,46.239624,10049000,PG,-0.005774,-0.002366,-0.003407,0.000000,0.000000,-100.000000,2008-01-04,0
0,53.385068,53.814755,53.092104,53.228821,3620300,MMM,-0.019163,-0.007902,-0.011261,0.000000,0.000000,-100.000000,2008-01-04,0
0,36.512117,36.817428,35.617011,35.658642,3048900,TRV,-0.022049,-0.005463,-0.016585,0.389215,28.016925,-100.000000,2008-01-04,0
0,26.299863,26.876764,26.299863,26.410152,9550700,DIS,-0.018535,-0.008033,-0.010502,0.000000,0.000000,-100.000000,2008-01-04,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3752,531.669983,532.969971,523.650024,528.000000,3751000,UNH,-0.417645,-0.561892,0.144247,0.991349,49.782786,15.513895,2022-11-29,3752
3752,152.720001,154.020004,151.259995,151.679993,6398100,CRM,-0.493688,-0.601808,0.108121,0.964399,49.093839,-0.847816,2022-11-29,3752
3752,38.020000,38.369999,37.950001,38.340000,14008800,VZ,0.272868,0.201219,0.071649,1.096433,52.299936,-34.559979,2022-11-29,3752


Set stock_df to be the final dataframe

In [119]:
stock_df = df_a.copy()

# 4. Design the RL Environment

Use OpenAI's Gym framework to build the environment.

In [130]:
day = stock_df.loc[0,:]

indicator_list=['macd', 'rsi', 'cci']

state = (
    [10000]
    + day.close.values.tolist()
    + [0]*28
    + sum((day[tech].values.tolist() for tech in indicator_list), [])
)

state = (
    [10000]
    + day.close.tolist()
    + [0]*28
    + sum(([day[tech]] for tech in indicator_list), [])
)

state[28]

12.804293632507324

In [ ]:
import gym
from gym import spaces

!pip install stable_baselines3
from stable_baselines3 import DQN

class StockTradingEnv(gym.Env):
    def __init__(self,
               df,
               num_stocks,
               tmax,
               initial_bal,
               state_space_dim,
               action_space_dim,
               tech_indicators,
               fractional_shares=False,
               day=0,
               initial=True):
        self.df = df
        self.num_stocks = num_stocks
        self.tmax = tmax    # set parameter that defines maximum number of shares that can be bought and sold
        self.initial_bal = initial_bal
        self.state_space_dim = state_space_dim
        self.action_space_dim = action_space_dim
        self.tech_indicators = tech_indicators
        self.fractional_shares = fractional_shares  # indicates if shares bought/sold can be fractional
        self.day = day

        self.data = self.df.loc[self.day, :]

        self.state = self._create_state()
        self.action_space = spaces.Box(low=-1, high=1, shape=(self.action_space_dim,))

        self.actions_history = []    # list to hold all actions taken in episode
        self.asset_history = []  # list to keep track of total asset value at each step
        self.date_history = []
        self.rewards_history = []
        self.state_history = []

        # values for reporting at end
        self.trades = 0 # count total trades

    def _create_state(self):
        """
        Function to create the beginning state for the system

        The state is a 3 column vector:
            Remaining balance
            Current price of each stock (float * num_stocks)
            Shares held of each stock (float * num_stocks)
            Technical indicators (len(tech_indicators) * num_stocks)
        """
        if self.initial:
            state = (
                [self.initial_bal]
                + [self.data.close.values.tolist()]
                + [0]*self.num_stocks
                + sum((self.data[tech].values.tolist() for tech in self.tech_indicators), [])
            )

        return state

    def _update_state(self):
        state = (
            [self.state[0]]
            + [self.data.close.values.tolist()]
            + list(self.state[(self.num_stocks + 1) : (self.stateself.num_stocks * 2 + 1)])
            + sum((self.data[tech].values.tolist() for tech in self.tech_indicators), [])
        )

        return state

    def _sell_stock(self, index, action):
        """
            Method to sell a stock

            Check if the we currently have any of the stock.
        """
        if self.state[self.num_stocks + index + 1] > 0:
            # sell only if we have any of the stock
            num_shares_sell = min(
                abs(action), self.state[self.num_stocks + index + 1]
            )   # can sell at most the number of shares we currently own
            sell_value = (self.state[index + 1] * num_shares_sell)
            self.state[0] += sell_value

            self.state[index + self.num_stocks + 1] -= num_shares_sell
            self.trades += 1
        else:
            num_shares_sell = 0

        return num_shares_sell()

    def _buy_stocks(self, index, action):
        if self.state[index + 1] > 0:
            # only buy if there is an actual buy price
            amount_avail = self.state[0] // self.state[index + 1]
        
            num_shares_buy = min(amount_avail, action)  # can buy max of amount_avail
            buy_value = (self.state[index + 1] * num_shares_buy)
            self.state[0] -= buy_value

            self.state[index + self.num_stocks + 1] += num_shares_buy
        else:
            num_shares_buy = 0

        return num_shares_buy

    def _asset_total(self):
        total_asset_val = self.state[0] + sum(
                np.array(self.state[1 : (self.num_stocks + 1)]) *
                np.array(self.state[(1 + self.num_stocks) : (self.num_stocks *2 + 1)])
            )
        return total_asset_val

    def step(self, actions):
        # check for terminal state
        self.terminal = self.day >= len(self.df.index.unique()) - 1

        if self.terminal:
            # calculate the total value of assets at end of episode
            final_total_assets = self._asset_total()
            total_reward = final_total_assets - self.initial_bal    # total gain

            return self.state, self.reward, self.terminal
        
        else:
            actions = actions * self.tmax   # scale {0,1} action
            if not self.fractional_shares:
                actions = actions.astype(int)   # can't buy fractional shares
            start_asset_total = self._asset_total()

            # sort actions small to big
            argsort_actions = np.argsort(actions)
            sell_inds = argsort_actions[: np.where(actions < 0)[0].shape[0]]
            buy_inds = argsort_actions[::-1][: np.where(actions > 0)[0].shape[0]]   # reverse list so biggest positive actions are first

            for ind in sell_inds:
                actions[ind] = self._sell_stock(ind, actions[ind]) * (-1)
            for ind in buy_inds:
                actions[ind] = self._buy_stocks(ind, actions[ind])

            self.actions_history.append(actions) # record actions

            # increment to next state
            self.day += 1
            self.data = self.df.loc[self.day, :]
            self._update_state()

            end_asset_total = self._asset_total()

            self.asset_history.append(end_asset_total)
            self.date_history.append(self._get_date())
            self.reward = self.end_asset_total - self.start_asset_total
            self.reward_history.append(self.reward)
            self.state_history.append(self.state)
            
        return self.state, self.reward, self.terminal, {}

    def reset(self):
        self.state = self._create_state()
        self.asset_history = [self.initial_bal]
        self.day = 0
        self.data = self.df.loc[self.day, :]
        

        







